# **Проект: Рынок видеоигр**

## **Загрузка и знакомство с данными**

In [12]:
# Загружаем необходимые библиотеки
import pandas as pd

In [13]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


### **Выводы о полученных данных**
Предоставлен датасет из 11 столбцов и 16956 строчек. 

Встречаются пропуски в нескольких столбцах: Name, Year of Release, Genre, EU sales, JP sales, Critic Score и User Score.
В некоторых случаях используются неверные типы данных: для года используется float64, тогда как оптимальнее будет работать с форматом int64. Для данных о количестве продаж (EU sales, JP sales, Other sales) и рейтингов (Critic Score, User Score) используется тип данных object, тогда как удобнее будет работать в формате float64. 

В дальнейшем необходимо будет перевести названия столбцов в snake case.


## **Проверка ошибок в данных и их предобработка**

### **Названия, или метки, столбцов датафрейма**

In [14]:
# Выводим оригинальные значения названий столбцов датафрейма для проверки стиля написания
print(f'Оригинальные названия столбцов: \n {df.columns.values}')

# Приводим названия столбцов к стилю snake case
df.columns = df.columns.str.lower().str.replace(' ', '_')

print(f'Новые названия столбцов: {df.columns.values}')

Оригинальные названия столбцов: 
 ['Name' 'Platform' 'Year of Release' 'Genre' 'NA sales' 'EU sales'
 'JP sales' 'Other sales' 'Critic Score' 'User Score' 'Rating']
Новые названия столбцов: ['name' 'platform' 'year_of_release' 'genre' 'na_sales' 'eu_sales'
 'jp_sales' 'other_sales' 'critic_score' 'user_score' 'rating']


### **Типы данных**

In [15]:
def check_df_col(df, col):
    '''
    Функция для вывода уникальных нечисловых значений в столбцах датафрейма
    '''
    # Преобразуем к числу значения в указанной колонке, некорректные значения станут NaN
    s_num = pd.to_numeric(df[col], errors='coerce')
    # Вытащим строки, где получилось NaN, и выведем уникальные исходные значения
    non_numeric_unique = df.loc[s_num.isna(), col].unique()
    print(f'Уникальные нечисловые значения для колонки {col}: {non_numeric_unique}')

check_df_col(df, 'jp_sales')
check_df_col(df, 'eu_sales')
check_df_col(df, 'user_score')

Уникальные нечисловые значения для колонки jp_sales: ['unknown']
Уникальные нечисловые значения для колонки eu_sales: ['unknown']
Уникальные нечисловые значения для колонки user_score: [nan 'tbd']


In [16]:
col_to_float64 = ['jp_sales', 'eu_sales', 'user_score']
df[col_to_float64] = df[col_to_float64].apply(pd.to_numeric, errors='coerce')

# Работаем с year_of_release: обрабатываем пропуски, чтобы привести этот столбец к типу int64
df['year_of_release'] = df['year_of_release'].fillna(0)
df['year_of_release'] = df['year_of_release'].astype('int32')


### **Наличие пропусков в данных**

In [17]:
# Считаем пропуски в абсолютных значениях
absolute_value = df.isna().sum()
print(f'Пропуски в абсолютных значениях:\n{absolute_value}\n')

# Считаем пропуски в относительных значениях
relative_value = (df.isna().sum()/ len(df) * 100).round(2)
print(f'Пропуски в относительных значениях:\n{relative_value}')


Пропуски в абсолютных значениях:
name                  2
platform              0
year_of_release       0
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

Пропуски в относительных значениях:
name                0.01
platform            0.00
year_of_release     0.00
genre               0.01
na_sales            0.00
eu_sales            0.04
jp_sales            0.02
other_sales         0.00
critic_score       51.39
user_score         54.66
rating             40.52
dtype: float64


### **Вывод о пропусках**

Было обнаружено несколько пропусков в столбцах name, genre, eu_sales, jp_sales, critic_score, user_score, rating.

Столбцы name и genre можно оставить как есть, т.е. не удалять строки с пропущенными значениями в этих столбцах: эти данные вряд ли будут использоваться при анализе.

Больше всего пропусков характерно для столбцов rating, user_score, critic_score. Эти пропуски могли возникнуть на этапе использования нескольких источников для создания датасета: некоторые из них могли не содержать информацию об оценках пользователей или критиков или не предполагать выставление оценки игре.

Их нельзя удалить, так как их слишком много. Нельзя выставить 0, так как это также может быть оценкой. Можно оставить эти пропуски, чтобы не искажать тренды и распределения.


In [18]:


# Вычисляем средние значения в группах eu_sales и jp_sales
jp_means = df.groupby(['platform', 'year_of_release'])['jp_sales'].mean()
eu_means = df.groupby(['platform', 'year_of_release'])['eu_sales'].mean()

# Создаем датафреймы со средними
jp_means_df = jp_means.reset_index()
jp_means_df.columns = ['platform', 'year_of_release', 'mean_jp_sales']
eu_means_df = eu_means.reset_index()
eu_means_df.columns = ['platform', 'year_of_release', 'mean_eu_sales']

# Объединяем с исходным датафреймом
df = df.merge(jp_means_df, on=['platform', 'year_of_release'], how='left')
df = df.merge(eu_means_df, on=['platform', 'year_of_release'], how='left')

# Заполняем пропуски средними значениями
df['jp_sales'] = df['jp_sales'].fillna(df['mean_jp_sales'])
df['eu_sales'] = df['eu_sales'].fillna(df['mean_eu_sales'])

# Удаляем вспомогательный столбец
df.drop('mean_jp_sales', axis=1, inplace=True)
df.drop('mean_eu_sales', axis=1, inplace=True)

### **Явные и неявные дубликаты в данных**

In [19]:
# Выводим уникальные значения в категориальных данных
unique_genre = df['genre'].unique()
unique_platform = df['platform'].unique()
unique_rating = df['rating'].unique()
unique_year = df['year_of_release'].unique()
print(f'Уникальные жанры: {unique_genre} \n' +
      f'Уникальные названия платформ: {unique_platform}\n' +
      f'Уникальные значения рейтинга: {unique_rating}\n' +
      f'Уникальные значения года выпуска: {unique_year}\n')

# Приводим названия жанров игр к нижнему регистру
df['genre'] = df['genre'].str.lower()
#print(f'Новые названия жанров: {df['genre'].unique()}\n')

# Приводим названия рейтинга к верхнему регистру
df['rating'] = df['rating'].str.upper()

# Считаем количество дубликатов
num_duplicates = df.duplicated().sum() 

# Считаем количество строк
num_rows = len(df)

# Считаем относительное значение удаленных дубликатов
relative_duplicates = (num_duplicates / num_rows).round(2)

# Удаляем дубликаты
df.drop_duplicates()
print(f'Количество явных дубликатов в абсолютном значении: {num_duplicates}\n' +
      f'Количество дубликатов в относительном значении: {relative_duplicates}')

Уникальные жанры: ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' nan 'MISC'
 'ROLE-PLAYING' 'RACING' 'ACTION' 'SHOOTER' 'FIGHTING' 'SPORTS' 'PLATFORM'
 'ADVENTURE' 'SIMULATION' 'PUZZLE' 'STRATEGY'] 
Уникальные названия платформ: ['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
Уникальные значения рейтинга: ['E' nan 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']
Уникальные значения года выпуска: [2006 1985 2008 2009 1996 1989 1984 2005 1999 2007 2010 2013 2004 1990
 1988 2002 2001 2011 1998 2015 2012 2014 1992 1997 1993 1994 1982 2016
 2003 1986 2000    0 1995 1991 1981 1987 1980 1983]

Количество явных дубликатов в абсолютном значении: 241
Количество дубликатов в относительном значении: 0.01


### **Вывод о работе с дубликатами**

Было подсчитано количество явных дубликатов: их было 241 (0,01 от общего числа данных). 

Для проверки на наличие неявных дубликатов были выведены уникальные значения категориальных данных. Были обнаружены дубликаты в столбце жанров. Названия были приведены к snake case, чтобы убрать их. 

## Промежуточный вывод по итогу предобработки данных 
Для работы дан датасет из 11 столбцов и 16956 строчек. Названия столбцов приведены к snake case.

Были выведены уникальные нечисловые значения для столбцов jp_sales, eu_sales и user_score, которые содержали данные типа object. 

Данные в столбце user_score были приведены к типу float64, а пропуски оставлены без изменений для того чтобы избежать искажения данных.

Пропуски в столбцах eu_sales и jp_sales были заменены средними значениями на основе года выпуска игры и платформы.

Пропуски в столбце year_of_release были заменены значением-индикатором 0 для перевода к int64.

Для дальнейшего анализа было удалено 241 явных дубликатов.

## **Фильтрация данных**

In [20]:
df_actual = df[(df['year_of_release'] >= 2000) & (df['year_of_release'] <= 2013)].copy()

## **Категоризация данных**

In [21]:
# Разделим все игры по оценкам пользователей
df_actual.loc[:, 'user_grade'] = pd.cut(df_actual['user_score'], bins=[0, 3, 8, 11], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'], right=False)

# Разделим все игры по оценкам критиков
df_actual.loc[:, 'critics_grade'] = pd.cut(df_actual['critic_score'], bins=[0, 30, 80, 101], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'], right=False)

# Сгруппируем по платформе и количеству игр, чтобы вывести топ-7 платформ
grouped_data = df_actual.groupby('platform')['name'].count()
sorted_data = grouped_data.sort_values(ascending=False)
top_platforms = sorted_data.head(7)
print(f'Топ-7 платформ по количеству игр, выпущенных за весь актуальный период:\n{top_platforms.to_string(header=False)}')


Топ-7 платформ по количеству игр, выпущенных за весь актуальный период:
PS2     2154
DS      2146
Wii     1294
PSP     1199
X360    1138
PS3     1107
GBA      826


In [22]:
# Добавляем категорию "нет оценки" для пользователей
df_actual['user_grade'] = df_actual['user_grade'].cat.add_categories('нет оценки').fillna('нет оценки')

# Добавляем категорию "нет оценки" для критиков
df_actual['critics_grade'] = df_actual['critics_grade'].cat.add_categories('нет оценки').fillna('нет оценки')

# Выводим результаты категоризации
print(f"Категоризации по оценке зрителей:\n{df_actual['user_grade'].value_counts().to_string(header=False)}\n")
print(f"Категоризации по оценке критиков:\n{df_actual['critics_grade'].value_counts().to_string(header=False)}\n")

Категоризации по оценке зрителей:
нет оценки        6408
средняя оценка    4148
высокая оценка    2307
низкая оценка      117

Категоризации по оценке критиков:
нет оценки        5713
средняя оценка    5500
высокая оценка    1712
низкая оценка       55



## **Основной вывод**

В ходе работы над датасетом была произведена предобработка данных. Названия столбцов были приведены к snake case. Были исправлены некорректные типы данных. 

Были обработаны пропуски: удалены или заменены на значения-индикаторы или приведено среднее значение.

В датасете были выявлены и удалены дубликаты. 

В результате работы были отобраны данные с 2000 по 2013 год включительно. 

Была проведена категоризация игр по оценкам пользователей и экспертов. Выделено три категории: высокая оценка, средняя оценка, низкая оценка. Категории отражены в новых столбцах ctitic_grade и user_grade.

Был составлен топ-7 платформ по количеству игр, выпущенных за необходимый для работы период. Наиболее популярной платформой является PS2.